In [6]:
#GDD calculation for corn from class CornGDD 2023. 
import pandas as pd

def fahrenheit_to_celsius(temp_f):
    return (temp_f - 32) * 5 / 9

def celsius_to_fahrenheit(temp_c):
    return (temp_c * 9 / 5) + 32

class CornGDD:
    def __init__(self, daily_temp_df, time_column, min_temp_column, max_temp_column, celsius=True):
        self.daily_temp_df = daily_temp_df
        self.time_column = time_column
        self.min_temp_column = min_temp_column
        self.max_temp_column = max_temp_column
        self.celsius = celsius

        # Ensure the time column is of datetime type
        self.daily_temp_df[self.time_column] = pd.to_datetime(self.daily_temp_df[self.time_column])

        # If initial data is in Celsius, optionally convert it to Fahrenheit for uniform GDD calculation
        if self.celsius:
            self.daily_temp_df[self.min_temp_column] = self.daily_temp_df[self.min_temp_column].apply(celsius_to_fahrenheit)
            self.daily_temp_df[self.max_temp_column] = self.daily_temp_df[self.max_temp_column].apply(celsius_to_fahrenheit)
            self.celsius = False  # Now all data is in Fahrenheit

    def con_gdd(self, start_date, end_date):
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)

        date_mask = (self.daily_temp_df[self.time_column] >= start_date) & (self.daily_temp_df[self.time_column] <= end_date)
        masked_data = self.daily_temp_df.loc[date_mask].copy()

        gdd_values = []
        cumulative_gdd = 0

        for index, row in masked_data.iterrows():
            min_temp = max(row[self.min_temp_column], 50)  # Fahrenheit base
            max_temp = min(row[self.max_temp_column], 86)  # Fahrenheit upper limit

            mean_temp = (min_temp + max_temp) / 2
            base_temp = 50  # Fahrenheit base temperature for GDD calculation
            gdd = max(mean_temp - base_temp, 0)

            gdd_values.append(gdd)
            cumulative_gdd += gdd

        masked_data['GDD'] = gdd_values
        masked_data['Cumulative_GDD'] = masked_data['GDD'].cumsum()

        return masked_data

# This modified class converts temperatures to Fahrenheit if they start in Celsius, then computes and returns GDD in Fahrenheit.


GDD for corn Becker 2023

In [7]:
import pandas as pd
Becker_GEMS = pd.read_csv("becker_gems_2023_2024.csv", on_bad_lines='skip')

In [8]:
Becker_GEMS_2023 = CornGDD(Becker_GEMS, time_column='datetime', min_temp_column='min_temp', max_temp_column='max_temp')

In [9]:
#Using year 2023 start date and end date for mine gdd
start_date = pd.to_datetime("5-17-2023").date()
end_date = pd.to_datetime("9-15-2023").date()

Cumulative GDD results for Corn Becker 2023

In [10]:
corn_gdd_BeckerGEMS = Becker_GEMS_2023.con_gdd(start_date, end_date)
print(corn_gdd_BeckerGEMS)

     clouds   datetime  dewpt  dhi  dni  ghi  max_dhi  max_dni  max_ghi  \
136      15 2023-05-17    7.5   56  450  338      120      919      940   
137      73 2023-05-18    8.0   56  452  340      120      919      941   
138      21 2023-05-19    2.8   56  453  341      120      919      943   
139       1 2023-05-20    3.6   56  454  342      120      920      944   
140       0 2023-05-21    4.7   56  455  343      121      920      946   
..      ...        ...    ...  ...  ...  ...      ...      ...      ...   
253      31 2023-09-11    8.6   45  367  241      112      884      771   
254      62 2023-09-12    9.3   44  365  238      112      883      766   
255      11 2023-09-13    5.8   44  363  236      112      882      761   
256      36 2023-09-14    9.3   44  360  234      111      881      756   
257      48 2023-09-15   11.7   43  358  231      111      880      751   

     max_temp  ...  t_solar_rad  temp          ts  wind_dir  wind_gust_spd  \
136     70.16  ...   